In [ ]:
import numpy as np
import pandas as pd
import re
import requests
import base64
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from hublib.ui import Download


api_url='http://localhost:8983/solr/strains_core/select?q=allele_symbol:\"'
api_url_two='http://localhost:8983/solr/strains_core/select?q=mp_ids:\"'
api_url_end='\"&rows=2000'

In [ ]:
class App():
    
    def __init__(self):
        self.tab = widgets.Tab()
        self.tab.children = []
        
        #fisrt tab
        self.table_container = widgets.Output()
        self.send_button, send_box = self.create_send_button()
        self.res_button = self.create_reset_button()
        self.container = widgets.VBox([send_box, widgets.HBox([self.send_button,self.res_button])])
        self.down_container = widgets.HBox([])
        self.tab_one = widgets.VBox([
            self.container,
            self.table_container,
            self.down_container
        ])
        
        #second tab
        self.table_container_two = widgets.Output()
        self.send_button_two, send_box_two = self.create_send_button_two()
        self.res_button_two = self.create_reset_button_two()
        self.container_two = widgets.VBox([send_box_two, widgets.HBox([self.send_button_two,self.res_button_two])])
        self.down_container_two = widgets.HBox([])
        self.tab_two = widgets.VBox([
            self.container_two,
            self.table_container_two,
            self.down_container_two
        ])
        
        #final view
        self.tab.children = [self.tab_one,self.tab_two]
        self.tab.set_title(0, 'MP terms by gene symbols')
        self.tab.set_title(1, 'gene symbols by MP terms')
        self.final_container = widgets.VBox([
            widgets.HTML(('<h1><em><b>Infrafrontier Strains API</b></em></h1>'), 
                         layout=widgets.Layout(margin='0 0 5em 0')),
            self.tab
        ])
        
    #FUNCTIONS FOR FIRST TAB PAGE
    
    
    def create_send_button(self):
        label = widgets.Label('Gene Symbol:')
        self.text_area = widgets.Textarea(placeholder='')
        send_b = widgets.Button(description='Submit', tooltip='submit your gene symbol', disabled=True)
        send_b.on_click(self.on_change_s)
        self.text_area.observe(self.on_change_text, names=['value'])
        sub_box = widgets.HBox([label,self.text_area])
        return(send_b, sub_box)
        
    def create_reset_button(self):
        reset_b = widgets.Button(description='Reset', tooltip='reset the environment to generate a new table', disabled=True)
        reset_b.on_click(self.on_click_r)
        return (reset_b) 
            
    def on_change_text(self,_):
        if (self.text_area.value == '') | (self.text_area.value.isspace()):
            self.send_button.disabled=True
        else:
            self.send_button.disabled=False
        
    def on_click_r(self,_):
        del(self.df)
        self.text_area.value=''
        self.res_button.disabled=True
        self.send_button.disabled=True
        remove = self.down_container.children[-1]
        self.down_container.children = self.down_container.children[:-1]
        remove.close()
        self.table_container.clear_output(wait=False)
        
    def on_change_s(self,_):
        with self.table_container:
            self.send_button.disabled = True
            gene = self.text_area.value
            req = requests.get(api_url+gene+api_url_end).json()["response"]["docs"]
            self.df = pd.DataFrame(columns=["Gene Symbol","Strain name","MP term ids", "MP term names"])

            sym=[]
            names = []
            mp_ids = []
            mp_terms = []
            for i in req:
                sym.append(i["allele_symbol"])
                names.append(i["name"])
                mp_ids.append(i["mp_ids"])
                mp_terms.append(i["mp_terms"])

            self.df["Gene Symbol"] = sym
            self.df["Strain name"] = names
            self.df["MP term ids"] = mp_ids
            self.df["MP term names"] = mp_terms
            del(sym,names,mp_ids,mp_terms,req)
            
            #create a copy of the dataframe to better display the table
            df2 = self.df.copy()
            for i in df2["MP term ids"]:
                i[0] = i[0].replace(",",", ")
            df2 = df2.applymap(lambda x: re.sub("['\[\]]","",str(x)))
            
            display(HTML(df2.to_html(justify="left", index=False)))
            self.csv_file = self.df.to_csv("result_table.csv",index=False)
            self.res_button.disabled = False
            htmlWidget = widgets.HTML(value="")
            self.create_download_link("result_table.csv", htmlWidget)
            self.down_container.children = tuple(list(self.down_container.children) + [htmlWidget])
            
    def create_download_link(self, filename, htmlWidget):  
        title="Click here to download the table in csv format"
        
        data = open(filename, "rb").read()
        b64 = base64.b64encode(data)
        payload = b64.decode()
        
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank"><button class="button-style">Download table as csv file</button></a>'
        htmlWidget.value = html.format(payload=payload,title=title,filename=filename)

        
    #FUNCTIONS FOR SECOND TAB PAGE
        
    def create_send_button_two(self):
        label = widgets.Label('MP term id:')
        self.text_area_two = widgets.Textarea(placeholder='')
        send_b = widgets.Button(description='Submit', tooltip='submit your MP term id', disabled=True)
        send_b.on_click(self.on_change_s_two)
        self.text_area_two.observe(self.on_change_text_two, names=['value'])
        sub_box = widgets.HBox([label,self.text_area_two])
        return(send_b, sub_box)
        
    def create_reset_button_two(self):
        reset_b = widgets.Button(description='Reset', tooltip='reset the environment to generate a new table', disabled=True)
        reset_b.on_click(self.on_click_r_two)
        return (reset_b) 
            
    def on_change_text_two(self,_):
        if (self.text_area_two.value == '') | (self.text_area_two.value.isspace()):
            self.send_button_two.disabled=True
        else:
            self.send_button_two.disabled=False
        
    def on_click_r_two(self,_):
        del(self.df_two)
        self.text_area_two.value=''
        self.res_button_two.disabled=True
        self.send_button_two.disabled=True
        remove = self.down_container_two.children[-1]
        self.down_container_two.children = self.down_container_two.children[:-1]
        remove.close()
        self.table_container_two.clear_output(wait=False)
        
        
    def on_change_s_two(self,_):
        with self.table_container_two:
            self.send_button_two.disabled = True
            term = self.text_area_two.value
            req = requests.get(api_url_two+term+api_url_end).json()["response"]["docs"]
            self.df_two = pd.DataFrame(columns=["MP term id", "Genes"])

            sym=[]
            for i in req:
                sym.append(i["allele_symbol"][0])

            sym = list(dict.fromkeys(sym))
            self.df_two["MP term id"] = [term]
            self.df_two["Genes"][0] = sym
            del(sym,req)
            
            #create a copy of the dataframe to better display the table
            df2_two = self.df_two.copy()
            df2_two = df2_two.applymap(lambda x: re.sub("['\[\]]","",str(x)))
            
            display(HTML(df2_two.to_html(justify="left", index=False)))
            self.csv_file_two = self.df_two.to_csv("result_table.csv",index=False)
            self.res_button_two.disabled = False
            htmlWidget_two = widgets.HTML(value="")
            self.create_download_link_two("result_table.csv", htmlWidget_two)
            self.down_container_two.children = tuple(list(self.down_container_two.children) + [htmlWidget_two])
            
    def create_download_link_two(self, filename, htmlWidget):  
        title="Click here to download the table in csv format"
        
        data = open(filename, "rb").read()
        b64 = base64.b64encode(data)
        payload = b64.decode()
        
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank"><button class="button-style">Download table as csv file</button></a>'
        htmlWidget.value = html.format(payload=payload,title=title,filename=filename)
        
    
app = App()
app.final_container